In [ ]:
import os
import sys
import re

os.environ["LITELLM_MODEL_NAME"] = "local/gemma"
os.environ["LITELLM_MODEL_API_BASE"] = "http://localhost:8000"
sys.path.append(os.path.abspath("../utils"))
sys.path.append(os.path.abspath("../outputs"))


In [ ]:
from litellm import CustomLLM, completion, get_llm_provider
from typing import List, Dict
from customlitellm import MyGemmaLLM

In [ ]:
# Instantiate your custom handler
import litellm


my_custom_llm = MyGemmaLLM()

# Register with LiteLLM
litellm.custom_provider_map = [
    {"provider": "custom_gemma_local", "custom_handler": my_custom_llm}
]

In [ ]:
def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="custom_gemma_local/gemma",  # Match this to what we define in config
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content

In [ ]:
def extract_code_block(response: str) -> str:
    """Extract the first code block from a markdown-style response."""
    if "```" not in response:
        return response.strip()

    parts = response.split("```")
    
    # Find the first code block (assumes code is in odd indexes)
    for i in range(1, len(parts), 2):
        code_block = parts[i].strip()
        # Remove language identifier if present
        if code_block.startswith("python"):
            code_block = code_block[len("python"):].strip()
        return code_block  # Return first code block only

    # If for some reason no block is found, return original
    return response.strip()


In [ ]:
def extract_function_name(code: str) -> str:
    """Extract the function name from Python function code."""
    match = re.search(r"def\s+(\w+)\s*\(", code)
    return match.group(1) if match else "unknown_function"


def develop_custom_function():
   # Get user input for function description
   print("\nWhat kind of function would you like to create?")
   print("Example: 'A function that calculates the factorial of a number'")
   print("Your description: ", end='')
   function_description = input().strip()

   # Initialize conversation with system prompt
   messages = [
      {"role": "system", "content": "You are a Python expert helping to develop a function."}
   ]

   # First prompt - Basic function
   messages.append({
      "role": "user",
      "content": f"Write a Python function that {function_description}. Output the function in a ```python code block```."
   })
   initial_function = generate_response(messages)

   # Parse the response to get the function code
   initial_function = extract_code_block(initial_function)

   print("\n=== Initial Function ===")
   print(initial_function)

   # Add assistant's response to conversation
   # Notice that I am purposely causing it to forget its commentary and just see the code so that
   # it appears that is always outputting just code.
   messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"})

   # Second prompt - Add documentation
   messages.append({
      "role": "user",
      "content": "Add comprehensive documentation to this function, including description, parameters, "
                 "return value, examples, and edge cases. Output the function in a ```python code block```."
   })
   documented_function = generate_response(messages)
   documented_function = extract_code_block(documented_function)
   print("\n=== Documented Function ===")
   print(documented_function)

   # Add documentation response to conversation
   messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+documented_function+"\n\n\`\`\`"})

   # Third prompt - Add test cases
   messages.append({
      "role": "user",
      "content": "Add unittest test cases for this function, including tests for basic functionality, "
                 "edge cases, error cases, and various input scenarios. Output the code in a \`\`\`python code block\`\`\`."
   })
   test_cases = generate_response(messages)
   # We will likely run into random problems here depending on if it outputs JUST the test cases or the
   # test cases AND the code. This is the type of issue we will learn to work through with agents in the course.
   test_cases = extract_code_block(test_cases)
   print("\n=== Test Cases ===")
   print(test_cases)

   # 4. Extract function name
   function_name = extract_function_name(documented_function)

   # 5. Create folder
   folder_path = os.path.join("..", "outputs", function_name)
   os.makedirs(folder_path, exist_ok=True)

   # 6. Save function code
   function_file = os.path.join(folder_path, f"{function_name}.py")
   with open(function_file, 'w') as f:
      f.write(documented_function)

   # 7. Save test code
   test_file = os.path.join(folder_path, f"test_{function_name}.py")
   with open(test_file, 'w') as f:
      f.write(test_cases)

   return documented_function, test_cases


In [ ]:
function_code, tests = develop_custom_function()
